In [138]:
import pandas as pd
train=pd.read_csv('../data/titanic_train.csv') #read train data
test=pd.read_csv('../data/titanic_test.csv') #read test data
print(train.info())  #show the information about train data,including counting values of null

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [139]:
print(train.isnull().sum()  )
c=train.Cabin.value_counts() #get the value of Cabin
c.head()
train.drop(labels='Cabin',axis=1)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C


In [140]:
train.Embarked=train.Embarked.fillna('S')
Em=train.Embarked.value_counts()
print(Em)

S    646
C    168
Q     77
Name: Embarked, dtype: int64


In [141]:
import re
train['cc']=train.Name.map(lambda x: str(re.compile(r',(.*)\.').findall(x)))#获取名字中的简称字样Mr,Miss,Mrs,Master,Dr等值
#替换上面的写法：train['cc']=train['Name'].apply(lambda x:x.split(',')[1].split('.')[0].strip())
c=train.loc[:,['cc','Age']].query('Age>0').groupby('cc').mean()  #按照名称辅助列看下各年龄的均值

In [142]:
train['Age']=train['Age'].fillna(0)#先对缺失值进行0填充
for i in range(1,891):
    if train['Age'][i]==0 and train['cc'][i]=="[' Mr']":
        train.loc[i, 'Age']=32
    if train['Age'][i]==0 and train['cc'][i] =="[' Mrs']":
        train.loc[i, 'Age']= 35
    if train['Age'][i]==0 and train['cc'][i] == "[' Miss']":
        train.loc[i, 'Age']=20
    if train['Age'][i]==0 and train['cc'][i] == "[' Master']":
        train.loc[i, 'Age']= 4
    if train['Age'][i]==0 and train['cc'][i] == "[' Dr']":
        train.loc[i,'Age']=42

In [143]:
categore=train.dtypes[train.dtypes=='object'].index

In [144]:
train
train=train.replace({'Sex':{'male':1,'female':2},
                     'Embarked':{'S':1,'C':2,'Q':3}}
                    )
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,cc
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,1,[' Mr']
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",2,38.0,1,0,PC 17599,71.2833,C85,2,[' Mrs']
2,3,1,3,"Heikkinen, Miss. Laina",2,26.0,0,0,STON/O2. 3101282,7.9250,NaN,1,[' Miss']
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",2,35.0,1,0,113803,53.1000,C123,1,[' Mrs']
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,1,[' Mr']
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1,27.0,0,0,211536,13.0000,NaN,1,[' Rev']
887,888,1,1,"Graham, Miss. Margaret Edith",2,19.0,0,0,112053,30.0000,B42,1,[' Miss']
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",2,20.0,1,2,W./C. 6607,23.4500,NaN,1,[' Miss']
889,890,1,1,"Behr, Mr. Karl Howell",1,26.0,0,0,111369,30.0000,C148,2,[' Mr']


In [145]:
data=pd.concat([train,test],keys=(['train','test']))

In [146]:
data=data.drop(labels=['cc','Name','Ticket','Cabin'],axis=1)
data.head()

PassengerId  Survived  Pclass Sex   Age  SibSp  Parch     Fare  \
train 0            1       0.0       3   1  22.0      1      0   7.2500   
      1            2       1.0       1   2  38.0      1      0  71.2833   
      2            3       1.0       3   2  26.0      0      0   7.9250   
      3            4       1.0       1   2  35.0      1      0  53.1000   
      4            5       0.0       3   1  35.0      0      0   8.0500   

        Embarked  
train 0        1  
      1        2  
      2        1  
      3        1  
      4        1

In [147]:
train_data=data.xs('train')#分开得到level 为train的测试数据
test_data=data.xs('test').drop(labels='Survived',axis=1)
x_train=train_data.drop(labels='Survived',axis=1)
y_train=train_data['Survived']
test_data=test_data.fillna(0)
test_data=test_data.replace({'Sex':{'male':1,'female':2},
                     'Embarked':{'S':1,'C':2,'Q':3}}
                    )

In [148]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [149]:
import numpy as np
S=StandardScaler()
S.fit(x_train)
x_train_stand=S.transform(x_train)
x_test_stand=S.transform(test_data)
Log=LogisticRegression(C=10)
Log.fit(x_train_stand,y_train)  #训练模型
prediction=Log.predict(x_test_stand)  #用训练的模型Log来预测测试数据
result=pd.DataFrame({'PassengerId':test_data.index,'Survived':prediction.astype(np.int32)})
#这里需要注意把prediction的数据转换成Int型不然系统判定不了，得分会为0
print(result)
result["PassengerId"]=result["PassengerId"]+892

     PassengerId  Survived
0              0         0
1              1         0
2              2         0
3              3         0
4              4         1
..           ...       ...
413          413         0
414          414         1
415          415         0
416          416         0
417          417         0

[418 rows x 2 columns]


In [150]:
val_data=pd.read_csv('../data/gender_submission.csv')
1-len(val_data.compare(result,keep_equal=False))/len(result)
# result

0.9114832535885168